In [ ]:
import requests
import pandas as pd
import time

# Función para obtener información general de la API de OMDB
def fetch_movie_data(api_key, search_query, num_pages=10):
    movies = []
    base_url = "http://www.omdbapi.com/"

    for page in range(1, num_pages + 1):
        params = {
            'apikey': api_key,
            's': search_query,
            'type': 'movie',
            'page': page
        }
        response = requests.get(base_url, params=params)
        data = response.json()

        if 'Search' in data:
            movies.extend(data['Search'])
        time.sleep(1)  # Para evitar exceder el límite de la API

    return movies

# Función para obtener información detallada de una película por imdbID
def fetch_movie_details(api_key, imdb_id):
    base_url = "http://www.omdbapi.com/"
    params = {
        'apikey': api_key,
        'i': imdb_id
    }
    response = requests.get(base_url, params=params)
    return response.json()

# API key y búsqueda
api_key = '9a208c2b'
search_query = 'movie'  # Ajusta según tus necesidades
num_pages = 10  # Ajusta según el número de resultados que quieras obtener

# Obtener datos generales
movies = fetch_movie_data(api_key, search_query, num_pages)

# Crear DataFrame y seleccionar solo las columnas relevantes
df = pd.DataFrame(movies)
df = df[['Title', 'Year', 'imdbID', 'Type']]  # Columnas seleccionadas: Título, Año, ID de IMDB y Tipo

# Agregar columnas 'Genre' y 'Rated' obteniendo detalles de cada película
genres = []
rated = []
for imdb_id in df['imdbID']:
    details = fetch_movie_details(api_key, imdb_id)
    genres.append(details.get('Genre', 'N/A'))
    rated.append(details.get('Rated', 'N/A'))
    time.sleep(1)  # Para evitar exceder el límite de la API

df['Genre'] = genres
df['Rated'] = rated

# Guardar a CSV
df.to_csv('peliculas.csv', index=False)
